There's been a Murder in SQL City!!

The detective gave us the crime scene report, but we somehow lost it. We vaguely remember that the crime was a ​**murder**​ that occurred sometime on **​15 Jan 2018​** and that it took place in **​SQL City**​. 

Let's use our Murder Mystery database to try to solve it!!!

In [25]:
# import libraries
import pandas as pd
import sqlite3 as sql

In [26]:
# setting up connection to the database
con = sql.connect('database/sql-murder-mystery.db')

Let's search through the crime scene reports using all information available:
* what:   murder
* when:   15 January 2018
* where:  SQL City 

In [27]:
# Checking crime scene reports
query_1 = '''
SELECT *
FROM crime_scene_report
WHERE city = "SQL City" AND date = "20180115" AND type = "murder";
'''

#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running our query
SQL_city_crime = pd.read_sql_query(query_1, con)
SQL_city_crime

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


Based on the crime note, there were two witnesses. Let’s look at their interview statements.

    Witness 1




In [28]:
witness_1 = '''
SELECT id, name, address_street_name, address_number, transcript
FROM person
INNER JOIN interview
ON person.id = interview.person_id
WHERE address_street_name == "Northwestern Dr" AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name = "Northwestern Dr");
'''

df = pd.read_sql_query(witness_1, con)
df

,id,name,address_street_name,address_number,transcript
0,14887,Morty Schapiro,Northwestern Dr,4919,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."



    Witness 2

In [29]:
witness_2 = '''
SELECT id, name, address_street_name, address_number, transcript
FROM person
INNER JOIN interview
on person.id = interview.person_id
WHERE name LIKE "%Annabel%" AND address_street_name = "Franklin Ave";
'''

w2 = pd.read_sql_query(witness_2, con)
w2

,id,name,address_street_name,address_number,transcript
0,16371,Annabel Miller,Franklin Ave,103,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


The second witness saw the murderer, and we'll examine her statement first to identify potential suspects.

In [30]:
suspects_q = '''
WITH ann_gym_times AS (
  SELECT
    ci.check_in_time  AS ann_in,
    ci.check_out_time AS ann_out
  FROM get_fit_now_check_in ci
  INNER JOIN get_fit_now_member mem
    ON ci.membership_id = mem.id
  WHERE ci.check_in_date = '20180109'
    AND mem.name = 'Annabel Miller'
)
SELECT mem.*, check_in_date, check_in_time, check_out_time
FROM get_fit_now_check_in ci
JOIN get_fit_now_member mem
  ON ci.membership_id = mem.id
CROSS JOIN ann_gym_times at
WHERE ci.check_in_date = '20180109'
  AND ci.check_out_time >= at.ann_in
  AND ci.check_in_time  <= at.ann_out;
'''

suspects = pd.read_sql_query(suspects_q, con)
suspects

,id,person_id,name,membership_start_date,membership_status,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,20180109,1530,1700
2,90081,16371,Annabel Miller,20160208,gold,20180109,1600,1700


Two other people were present in the gym when our witness (Annabel Miller) was there on 9 January 2018. Let’s examine the first witness’s statement for confirmation of the suspect.

In [31]:
suspect_q = '''
SELECT m.id, p.name, age, height, eye_color, hair_color, gender, plate_number, car_make, car_model
FROM get_fit_now_member AS m
JOIN person AS p
  ON m.person_id = p.id
JOIN drivers_license AS d
  ON p.license_id = d.id
WHERE m.id LIKE '%48Z%'
  AND m.membership_status = 'gold'
  AND d.plate_number LIKE '%H42W%';
'''

suspect = pd.read_sql_query(suspect_q, con)
suspect

,id,name,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,Jeremy Bowers,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


Our primary suspect has been identified as Jeremy Bowers.
But this is only the beginning…